<h1 style="color:red; font-weight:bold;">Fetch Coding Exercise - Analytics Engineer</h1>

1. Review unstructured JSON data and diagram a new structured relational data model

![Fetch ERD](ERD.png)

2. Generate a query that answers a predetermined business question

In [ ]:
with user_filter as (select distinct user_id
                     from users
                     where created_date between current_date - interval '180 days' and current_date),
     spend as (select b.name
                    , sum(ri.item_price) as total_spend
               from brands b
               left join receipt_items ri on b.brand_id = ri.brand_id
               left join receipts r on ri.receipt_id = r.receipt_id
               inner join user_filter u on r.user_id = u.user_id
               group by b.name),
     transactions as (select b.name
                           , sum(ri.item_quantity) as total_transactions
                      from brands b
                      left join receipt_items ri on b.brand_id = ri.brand_id
                      left join receipts r on ri.receipt_id = r.receipt_id
                      inner join user_filter u on r.user_id = u.user_id
                      group by b.name)
select 
    (select name from spend order by total_spend desc limit 1) as brand_with_most_spend,
    (select name from transactions order by total_transactions desc limit 1) as brand_with_most_transactions;

3. Generate a query to capture data quality issues against the new structured relational data model

In [1]:
import os
import json
import pandas as pd

In [2]:
# Define file paths
file_paths = ['brands.json', 'receipts.json', 'users.json']

# Loop through each file and load it into a DataFrame with the file name as the variable name
for file_path in file_paths:
    # Load the JSON data into a pandas DataFrame
    df = pd.read_json(file_path, orient='records', lines=True)
    
    # Use the file name to create the variable name
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Dynamically assign the DataFrame to a variable with the file name as the name
    globals()[file_name] = df

In [3]:
# 1. Check Data Types

print("Brands Data Types:")
print(brands.dtypes, "\n")

print("Receipts Data Types:")
print(receipts.dtypes, "\n")

print("Users Data Types:")
print(users.dtypes, "\n")

Brands Data Types:
_id              object
barcode           int64
category         object
categoryCode     object
cpg              object
name             object
topBrand        float64
brandCode        object
dtype: object 

Receipts Data Types:
_id                         object
bonusPointsEarned          float64
bonusPointsEarnedReason     object
createDate                  object
dateScanned                 object
finishedDate                object
modifyDate                  object
pointsAwardedDate           object
pointsEarned               float64
purchaseDate                object
purchasedItemCount         float64
rewardsReceiptItemList      object
rewardsReceiptStatus        object
totalSpent                 float64
userId                      object
dtype: object 

Users Data Types:
_id             object
active            bool
createdDate     object
lastLogin       object
role            object
signUpSource    object
state           object
dtype: object 



In [4]:
# 2. Check for Missing Values

print("Brands Missing Values:")
missing_brands = brands.isnull().sum()
print(missing_brands[missing_brands > 0], "\n")

print("Receipts Missing Values:")
missing_receipts = receipts.isnull().sum()
print(missing_receipts[missing_receipts > 0], "\n")

print("Users Missing Values:")
missing_users = users.isnull().sum()
print(missing_users[missing_users > 0], "\n")

Brands Missing Values:
category        155
categoryCode    650
topBrand        612
brandCode       234
dtype: int64 

Receipts Missing Values:
bonusPointsEarned          575
bonusPointsEarnedReason    575
finishedDate               551
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
totalSpent                 435
dtype: int64 

Users Missing Values:
lastLogin       62
signUpSource    48
state           56
dtype: int64 



In [5]:
# 3. Check for Duplicate Values on assumed PK columns

print("Duplicated _id in Users Data:")
duplicate_users_ids = users[users.duplicated(subset=['_id'], keep=False)]
print(duplicate_users_ids[['state', 'createdDate', 'role', 'active', '_id']])  # Show relevant columns

print("\nDuplicated _id in Brands Data:")
duplicate_brands_ids = brands[brands.duplicated(subset=['_id'], keep=False)]
print(duplicate_brands_ids[['name', 'barcode', 'category', 'topBrand', '_id']])  # Show relevant columns

print("\nDuplicated _id in Receipts Data:")
duplicate_receipts_ids = receipts[receipts.duplicated(subset=['_id'], keep=False)]
print(duplicate_receipts_ids[['userId', 'purchaseDate', 'totalSpent', 'purchasedItemCount', '_id']])  # Show relevant columns

Duplicated _id in Users Data:
    state               createdDate         role  active  \
0      WI  {'$date': 1609687444800}     consumer    True   
1      WI  {'$date': 1609687444800}     consumer    True   
2      WI  {'$date': 1609687444800}     consumer    True   
3      WI  {'$date': 1609687530554}     consumer    True   
4      WI  {'$date': 1609687444800}     consumer    True   
..    ...                       ...          ...     ...   
490   NaN  {'$date': 1418998882381}  fetch-staff    True   
491   NaN  {'$date': 1418998882381}  fetch-staff    True   
492   NaN  {'$date': 1418998882381}  fetch-staff    True   
493   NaN  {'$date': 1418998882381}  fetch-staff    True   
494   NaN  {'$date': 1418998882381}  fetch-staff    True   

                                      _id  
0    {'$oid': '5ff1e194b6a9d73a3a9f1052'}  
1    {'$oid': '5ff1e194b6a9d73a3a9f1052'}  
2    {'$oid': '5ff1e194b6a9d73a3a9f1052'}  
3    {'$oid': '5ff1e1eacfcf6c399c274ae6'}  
4    {'$oid': '5ff1e194b6a9d7

In [6]:
# 4. Check Summary Statistics for numerical columns

print("Brands Summary Statistics:")
print(brands.describe(), "\n")

print("Receipts Summary Statistics:")
print(receipts.describe(), "\n")

print("Users Summary Statistics:")
print(users.describe(), "\n")

Brands Summary Statistics:
            barcode    topBrand
count  1.167000e+03  555.000000
mean   5.111115e+11    0.055856
std    2.874497e+05    0.229850
min    5.111110e+11    0.000000
25%    5.111112e+11    0.000000
50%    5.111114e+11    0.000000
75%    5.111117e+11    0.000000
max    5.111119e+11    1.000000 

Receipts Summary Statistics:
       bonusPointsEarned  pointsEarned  purchasedItemCount   totalSpent
count         544.000000    609.000000           635.00000   684.000000
mean          238.893382    585.962890            14.75748    77.796857
std           299.091731   1357.166947            61.13424   347.110349
min             5.000000      0.000000             0.00000     0.000000
25%             5.000000      5.000000             1.00000     1.000000
50%            45.000000    150.000000             2.00000    18.200000
75%           500.000000    750.000000             5.00000    34.960000
max           750.000000  10199.800000           689.00000  4721.950000 

User

In [7]:
# 5. Check for Invalid or Future Dates:

# Convert date columns to datetime (if they are not already)
receipts['purchaseDate'] = pd.to_datetime(receipts['purchaseDate'], errors='coerce')
receipts['createDate'] = pd.to_datetime(receipts['createDate'], errors='coerce')

print("Receipts Data - Checking for Invalid or Future Dates:")

invalid_purchase_dates = receipts[receipts['purchaseDate'] > pd.Timestamp.today()]
invalid_create_dates = receipts[receipts['createDate'] > pd.Timestamp.today()]

print(f"Invalid future purchase dates: {invalid_purchase_dates[['purchaseDate', '_id']]}\n")
print(f"Invalid future create dates: {invalid_create_dates[['createDate', '_id']]}\n")

users['createdDate'] = pd.to_datetime(users['createdDate'], errors='coerce')
print("Users Data - Checking for Invalid 'createdDate' Values:")
invalid_user_creation_dates = users[users['createdDate'] > pd.Timestamp.today()]
print(f"Invalid future createdDate: {invalid_user_creation_dates[['createdDate', '_id']]}\n")

Receipts Data - Checking for Invalid or Future Dates:
Invalid future purchase dates: Empty DataFrame
Columns: [purchaseDate, _id]
Index: []

Invalid future create dates: Empty DataFrame
Columns: [createDate, _id]
Index: []

Users Data - Checking for Invalid 'createdDate' Values:
Invalid future createdDate: Empty DataFrame
Columns: [createdDate, _id]
Index: []



In [8]:
# 6. Check for Users with missing 'state' information

print("Users Data - Checking for Missing States:")
missing_states = users[users['state'].isnull()]
print(f"Users with missing state information: {missing_states[['_id', 'state']]}\n")

# Check for Inactive Users
print("Users Data - Checking for Inactive Accounts:")
inactive_users = users[users['active'] == False]
print(f"Inactive users: {inactive_users[['_id', 'active']]}\n")

Users Data - Checking for Missing States:
Users with missing state information:                                       _id state
344  {'$oid': '60145ff384231211ce796d51'}   NaN
350  {'$oid': '60145ff384231211ce796d51'}   NaN
375  {'$oid': '60186237c8b50e11d8454d5f'}   NaN
376  {'$oid': '60186237c8b50e11d8454d5f'}   NaN
378  {'$oid': '60186237c8b50e11d8454d5f'}   NaN
381  {'$oid': '60186237c8b50e11d8454d5f'}   NaN
382  {'$oid': '60186237c8b50e11d8454d5f'}   NaN
422  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
423  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
424  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
425  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
426  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
428  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
430  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
431  {'$oid': '5a43c08fe4b014fd6b6a0612'}   NaN
432  {'$oid': '5fbc35711d967d1222cbfefc'}   NaN
433  {'$oid': '5fbc35711d967d1222cbfefc'}   NaN
434  {'$oid': '5fbc35711d967d1222cbfefc'}   NaN
455  {'$

In [9]:
# 7. Check for Users with invalid roles

print("Users Data - Checking for Invalid Roles:")
invalid_roles = users[users['role'] != 'consumer']
print(f"Users with invalid roles: {invalid_roles[['_id', 'role']]}\n")

Users Data - Checking for Invalid Roles:
Users with invalid roles:                                       _id         role
395  {'$oid': '59c124bae4b0299e55b0f330'}  fetch-staff
396  {'$oid': '59c124bae4b0299e55b0f330'}  fetch-staff
397  {'$oid': '59c124bae4b0299e55b0f330'}  fetch-staff
398  {'$oid': '59c124bae4b0299e55b0f330'}  fetch-staff
399  {'$oid': '59c124bae4b0299e55b0f330'}  fetch-staff
..                                    ...          ...
490  {'$oid': '54943462e4b07e684157a532'}  fetch-staff
491  {'$oid': '54943462e4b07e684157a532'}  fetch-staff
492  {'$oid': '54943462e4b07e684157a532'}  fetch-staff
493  {'$oid': '54943462e4b07e684157a532'}  fetch-staff
494  {'$oid': '54943462e4b07e684157a532'}  fetch-staff

[82 rows x 2 columns]



In [10]:
# 8. Check for Missing Barcodes, Duplicate Barcodes, and Missing Brand Names

print("Brands Data - Checking for Missing or Duplicated Barcode:")
missing_barcodes = brands[brands['barcode'].isnull()]
print(f"Brands with missing barcode: {missing_barcodes[['barcode', '_id']]}\n")

duplicate_barcodes = brands[brands.duplicated(subset=['barcode'], keep=False)]
print(f"Brands with duplicate barcodes: {duplicate_barcodes[['barcode', '_id']]}\n")

missing_brand_names = brands[brands['name'].isnull()]
print(f"Brands with missing names: {missing_brand_names[['name', '_id']]}\n")

Brands Data - Checking for Missing or Duplicated Barcode:
Brands with missing barcode: Empty DataFrame
Columns: [barcode, _id]
Index: []

Brands with duplicate barcodes:            barcode                                   _id
9     511111504788  {'$oid': '5c408e8bcd244a1fdb47aee7'}
20    511111305125  {'$oid': '5c4699f387ff3577e203ea29'}
129   511111504139  {'$oid': '5a7e0604e4b0aedb3b84afd3'}
152   511111204923  {'$oid': '5c45f91b87ff3552f950f027'}
194   511111605058  {'$oid': '5d6415d5a3a018514994f429'}
299   511111504139  {'$oid': '5a8c33f3e4b07f0a2dac8943'}
412   511111504788  {'$oid': '5ccb2ece166eb31bbbadccbe'}
467   511111004790  {'$oid': '5c409ab4cd244a3539b84162'}
536   511111204923  {'$oid': '5d6027f46d5f3b23d1bc7906'}
651   511111305125  {'$oid': '5d642d65a3a018514994f42d'}
1002  511111704140  {'$oid': '5a8c344ae4b0ccf165fac9e5'}
1012  511111605058  {'$oid': '5c4637ba87ff35681e840d57'}
1015  511111704140  {'$oid': '5a7e0665e4b0aedb3b84afd4'}
1071  511111004790  {'$oid': '5c

In [11]:
# 9. Check for UserId inconsistencies (UserId not in Users DataFrame)
print("Receipts Data - Checking for Inconsistent User IDs:")
invalid_user_ids = receipts[~receipts['userId'].isin(users['_id'])]
print(f"Receipts with invalid userId: {invalid_user_ids[['userId', '_id']]}\n")

Receipts Data - Checking for Inconsistent User IDs:
Receipts with invalid userId:                         userId                                   _id
0     5ff1e1eacfcf6c399c274ae6  {'$oid': '5ff1e1eb0a720f0523000575'}
1     5ff1e194b6a9d73a3a9f1052  {'$oid': '5ff1e1bb0a720f052300056b'}
2     5ff1e1f1cfcf6c399c274b0b  {'$oid': '5ff1e1f10a720f052300057a'}
3     5ff1e1eacfcf6c399c274ae6  {'$oid': '5ff1e1ee0a7214ada100056f'}
4     5ff1e194b6a9d73a3a9f1052  {'$oid': '5ff1e1d20a7214ada1000561'}
...                        ...                                   ...
1114  5fc961c3b8cfca11a077dd33  {'$oid': '603cc0630a720fde100003e6'}
1115  5fc961c3b8cfca11a077dd33  {'$oid': '603d0b710a720fde1000042a'}
1116  5fc961c3b8cfca11a077dd33  {'$oid': '603cf5290a720fde10000413'}
1117  5fc961c3b8cfca11a077dd33  {'$oid': '603ce7100a7217c72c000405'}
1118  5fc961c3b8cfca11a077dd33  {'$oid': '603c4fea0a7217c72c000389'}

[1119 rows x 2 columns]



In [12]:
# 10. Check for missing or empty items in 'rewardsReceiptItemList'

print("Receipts Data - Checking for Missing or Empty Items in 'rewardsReceiptItemList':")
empty_items = receipts[receipts['rewardsReceiptItemList'].isnull() | (receipts['rewardsReceiptItemList'].str.len() == 0)]
print(f"Receipts with missing or empty item list: {empty_items[['rewardsReceiptItemList', '_id']]}\n")

Receipts Data - Checking for Missing or Empty Items in 'rewardsReceiptItemList':
Receipts with missing or empty item list:      rewardsReceiptItemList                                   _id
71                      NaN  {'$oid': '5ff475820a7214ada10005cf'}
93                      NaN  {'$oid': '5ff5ecb90a7214ada10005f9'}
149                     NaN  {'$oid': '5ff726860a720f05230005ec'}
175                     NaN  {'$oid': '5ff8da570a720f05c5000015'}
212                     NaN  {'$oid': '5ffce8570a7214ad4e003e6f'}
...                     ...                                   ...
1110                    NaN  {'$oid': '603c6adf0a720fde1000039a'}
1111                    NaN  {'$oid': '603c9e6e0a720fde100003c7'}
1115                    NaN  {'$oid': '603d0b710a720fde1000042a'}
1116                    NaN  {'$oid': '603cf5290a720fde10000413'}
1118                    NaN  {'$oid': '603c4fea0a7217c72c000389'}

[440 rows x 2 columns]



4. Write a short email or Slack message to the business stakeholder

Hello [Stakeholder Name],

During a recent review of our datasets, I’ve identified several key data quality issues which need to be addressed. There are missing values in critical columns across Brands, Receipts, and Users. For example, several Brands entries are missing values for category, brand code, and top brand status, while Receipts is missing important fields like purchase dates, points awarded, and total spent amounts. Additionally, Users have missing data in key columns such as state and last login, and there are some duplicate user IDs.

We also have some data integrity issues, such as invalid or missing userIds in the Receipts dataset and missing or empty item lists for certain receipts. Lastly, there are inconsistencies with user roles in the Users table that need to be addressed.

These issues may impact the accuracy of our analysis and reporting. To move forward, we need to understand whether missing values should be imputed, or if they represent known data gaps. For the duplicate Users records, we’ll need to clarify how to handle these duplicates and ensure that user identities remain unique.

I recommend we prioritize fixing these issues to maintain data quality, which will help us avoid errors in reporting and insights. Please let me know if you’d like more details on any of these issues or if you have suggestions on how to address them.

Best,
[Name]